In [ ]:
!pip install langchain
!pip install typing-inspect==0.8.0 typing_extensions==4.5.0
!pip install pydantic==1.10.11
# !pip install llama-cpp-python
# !pip install tensorflow
!pip install accelerate
!pip install bitsandbytes
!pip install -U tokenizers
!pip install transformers
!pip install sentence-transformers
!pip install faiss-cpu
!pip install chromadb

In [ ]:
from langchain.docstore.document import Document 
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS,Chroma

class vectorstore:
    def __init__(self,sheet_name):
        self.sheet_name = sheet_name
        self.df = pd.read_excel('All_Files.xlsx',sheet_name = sheet_name)
        self.df = self.df.fillna('No Data')
        self.df = self.df.astype(str)
        
        self.columns_to_embed = self.df.columns.values
        self.columns_to_metadata = self.df.columns.values

        self.df_columns_to_embed=self.df[self.columns_to_embed]
        self.df_columns_to_metadata=self.df[self.columns_to_metadata]

        self.to_embed_dict=self.df_columns_to_embed.to_dict('records')
        self.to_metadata_dict=self.df_columns_to_metadata.to_dict('records')
   
    def load_documents(self):
        # Define the columns we want to embed vs which ones we want in metadata
        self.docs=[]
        for i in range(len(self.to_embed_dict)):
            to_embed = self.to_embed_dict[i]
            to_metadata = self.to_metadata_dict[i]
            to_embed = "\n".join(f"{k.strip()}: {v.strip()}" for k, v in to_embed.items())
            newDoc=Document(page_content=to_embed,metadata=to_metadata)
            self.docs.append(newDoc)

    def document_splitter(self):
        text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
        self.docs = text_splitter.split_documents(self.docs)

    def save_embedding(self):
        self.hf = HuggingFaceEmbeddings()
        self.db = FAISS.from_documents(self.docs, self.hf)
        self.db.save_local(f"faiss_index_{self.sheet_name}")
        print(f'{self.sheet_name} saved')

    def embedding_load_and_search(self,query,num_of_matches):
        self.hf = HuggingFaceEmbeddings()
        self.db = FAISS.load_local(f"faiss_index_{self.sheet_name}", self.hf)
        df_result = pd.DataFrame(data=[],columns=self.columns_to_metadata)

        results = self.db.similarity_search(query,k=num_of_matches)
        for i in range (len(results)):
            df_embed_result = pd.DataFrame([results[i].metadata])
            df_result = pd.concat([df_result,df_embed_result],axis=0,ignore_index=True)

        return df_result
    
    def load_split_save(self):
        self.load_documents()
        self.document_splitter()
        self.save_embedding()
        